In [10]:
import SimpleITK as sitk
import numpy as np

def dice(a,b):
    inter = np.logical_and(a>0, b>0).sum()
    return 2*inter / (a.sum() + b.sum())

ct = sitk.ReadImage(r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\CT_WHOLE_CNS_cropped.nii")
seg = sitk.ReadImage(r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\mandible.nii.gz")

# rotate by +5 degrees around z-axis
tx = sitk.Euler3DTransform()
tx.SetRotation(0, 0, np.deg2rad(5))
tx.SetCenter(np.array(ct.GetSize())/2 * np.array(ct.GetSpacing())/2)

# apply transform to CT
resampled_ct = sitk.Resample(ct, ct, tx, sitk.sitkLinear, 0.0, ct.GetPixelID())
sitk.WriteImage(resampled_ct, r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\resampled\CT_WHOLE_CNS_cropped_resampled_transformed.nii.gz")


In [3]:
from totalsegmentator.python_api import totalsegmentator

def main():
    input_path=r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\Resampled\CT_WHOLE_CNS_cropped_resampled_transformed.nii.gz"
    output_path= r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\resampled"

    totalsegmentator(input_path,output_path,task = 'craniofacial_structures')
    print('\nDone!')

if __name__ == "__main__":
 main()

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough segmentation for cropping...
Resampling...
  Resampled in 1.38s
Predicting...


d:\Users\Abigail Crowther\Desktop\Uni\Year 4\MPhys Projects\Semester 1 - Modelling Facial Deformation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:01<00:00,  1.77s/it]


  Predicted in 25.76s
Resampling...
  cropping from (312, 312, 150) to (189, 233, 118)
Resampling...
  Resampled in 35.00s
Predicting...


100%|██████████| 200/200 [1:09:52<00:00, 20.96s/it]


  Predicted in 4302.21s
Resampling...
Saving segmentations...
  Saved in 57.74s

Done!


In [11]:
# apply same transform to original segmentation
resampled_seg = sitk.Resample(seg, seg, tx, sitk.sitkNearestNeighbor, 0, seg.GetPixelID())
sitk.WriteImage(resampled_seg, r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\resampled\mandible_transformed.nii.gz")

# compare Dice between resampled segmentation and transformed segmentation
seg_pred = sitk.ReadImage(r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\Resampled\mandible.nii.gz")
arr1 = sitk.GetArrayFromImage(resampled_seg)
arr2 = sitk.GetArrayFromImage(seg_pred)
print("Overall Dice:", dice(arr1>0, arr2>0))

Overall Dice: 0.9567009650412219


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\Resampled\CT_WHOLE_CNS_cropped_resampled_transformed.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    "mandible_resampled": r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\Resampled\mandible.nii.gz",
    "mandible_transformed": r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\Resampled\mandible_transformed.nii.gz",
}

segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    "mandible_resampled": (0.2, 0.4, 1.0, 0.25),          # blueish
    "mandible_transformed": (1.0, 0.6, 0.0, 0.35),      # orange
}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0Q00Q910"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=75, description='Slice:', layout=Layout(width='80%'), max=149), Dropdown…

<function __main__.show_slice(slice_index, plane, alpha=0.35, patient_id='UIDQQ0Q00Q910')>